Cloning into 'Recommender-Systems-Challenge'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 1098 (delta 105), reused 99 (delta 55), pack-reused 937
Receiving objects: 100% (1098/1098), 208.89 MiB | 36.46 MiB/s, done.
Resolving deltas: 100% (643/643), done.
Checking out files: 100% (250/250), done.


In [ ]:
!python run_compile_all_cython.py

run_compile_all_cython: Found 11 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/usr/bin/python3'
Compiling [1/11]: MatrixFactorization_Cython_Epoch.pyx... 
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/MatrixFactorization/Cython/MatrixFactorization_Cython_Epoch.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
In file included from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1822:0,
                 from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from MatrixFactorization_Cython_Epoch.c:610:
/usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/npy_1_7_d

In [ ]:
pip install optuna

     |████████████████████████████████| 266kB 9.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 7.5MB/s 
     |████████████████████████████████| 163kB 15.5MB/s 
     |████████████████████████████████| 133kB 14.6MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 112kB 16.9MB/s 
     |████████████████████████████████| 81kB 8.1MB/s 
  Created wheel for optuna: filename=optuna-2.3.0-cp36-none-any.whl size=359761 sha256=d78df6ac7dc36681cc0cec0a8c55b18eaae14154db71055789bc1622e6ecd32f
  Stored in directory: /root/.cache/pip/wheels/fa/91/19/64b0ec6b964f89c0695a9dc6db6f851d0b54c5381a5c9cadfb
Successfully built optuna
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp36-none-any.whl size=13702 sha256=dc501464d3c4407bdbf10f325065b1d41bacc619482988a506df550c1e66a460
  Stored in directory: /root/.cache/pip/whe

In [ ]:
pip install implicit

     |████████████████████████████████| 1.1MB 9.6MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419480 sha256=1f7e6e7c7c742d4ad9003d0b693b07d7b8ba6f3db2035dd5edb22aea5099077d
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np 
import scipy.sparse as sps
import os
import sys
import matplotlib.pyplot as plt
import optuna

In [ ]:
dataset = pd.read_csv('/content/New Challenge Data/data_train.csv')
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


In [ ]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

test_users = pd.read_csv('/content/New Challenge Data/data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
7939,7942
7940,7943
7941,7944
7942,7945


In [ ]:
ICM_df = pd.read_csv('/content/New Challenge Data/data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [ ]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [ ]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Base.Evaluation.Evaluator import EvaluatorHoldout

URM_train_1, URM_validation_1 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 1)

URM_train_2, URM_validation_2 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 2)

URM_train_3, URM_validation_3 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 3)

URM_train_4, URM_validation_4 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 4)

URM_train_5, URM_validation_5 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 5)

URM_train_vector = [URM_train_1,URM_train_2,URM_train_3,URM_train_4,URM_train_5]

evaluator_validation_1 = EvaluatorHoldout(URM_validation_1, cutoff_list=[10])
evaluator_validation_2 = EvaluatorHoldout(URM_validation_2, cutoff_list=[10])
evaluator_validation_3 = EvaluatorHoldout(URM_validation_3, cutoff_list=[10])
evaluator_validation_4 = EvaluatorHoldout(URM_validation_4, cutoff_list=[10])
evaluator_validation_5 = EvaluatorHoldout(URM_validation_5, cutoff_list=[10])

Evaluator_vector = [evaluator_validation_1, evaluator_validation_2, evaluator_validation_3, evaluator_validation_4, evaluator_validation_5]

In [ ]:
from GraphBased.RP3betaRecommender import RP3betaRecommender
from MatrixFactorization.IALSRecommender_implicit import IALSRecommender_implicit
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from KNN.ItemKNNScoresHybridNRecommender import ItemKNNScoresHybridNRecommender
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython


In [ ]:
class Objective(object):
    def __init__(self, URM_train_vector, ICM_all, Evaluator_vector):
        # Hold this implementation specific arguments as the fields of the class.
        self.URM_train_vector = URM_train_vector
        self.ICM_all = ICM_all
        self.Evaluator_vector = Evaluator_vector
        self.MAP_vector = np.zeros(5)

        

    def __call__(self, trial):
        # Calculate an objective value by using the extra arguments.
        alpha = trial.suggest_uniform('alpha', 0.1, 0.9)
        beta = trial.suggest_uniform('beta', 0.1, 0.9)
        gamma = trial.suggest_uniform('gamma', 0.1, 0.9)
        phi = trial.suggest_uniform('phi', 0.1, 0.9)
        self.MAP_vector = np.zeros(5)
        i = 0
        j=1
        for URM_train, evaluator_validation in zip(self.URM_train_vector,self.Evaluator_vector):
            
            URM_train_ICM_all_IALS = sps.vstack([URM_train*(1-0.5293971458364172), ICM_all.T*0.5293971458364172])
            recommender_IALS = IALSRecommender_implicit(URM_train_ICM_all_IALS)
            filename_IALS = 'MF_IALS_fold_number' + str(j)
            recommender_IALS.load_model('/content/drive/MyDrive/Rec_sys_models/MF_saved_2/', file_name=filename_IALS)
            
            URM_train_ICM_all_SLIM = sps.vstack([URM_train*(1-0.421), ICM_all.T*0.421])
            recommender_SLIM = SLIMElasticNetRecommender(URM_train_ICM_all_SLIM)
            filename_SLIM = 'SLIM_EN_fold_number' + str(j)
            recommender_SLIM.load_model('/content/drive/MyDrive/Rec_sys_models/SLIM_saved/', file_name=filename_SLIM)
            recommender_SLIM.W_sparse.data = np.power(recommender_SLIM.W_sparse.data, 0.8546)

            URM_train_ICM_all_RP3 = sps.vstack([URM_train, ICM_all.T])
            recommender_RP3Beta = RP3betaRecommender(URM_train_ICM_all_RP3)
            filename_RP3Beta = 'RP3_fold_number' + str(j)
            recommender_RP3Beta.load_model('/content/drive/MyDrive/Rec_sys_models/RP3_saved_2/', file_name=filename_RP3Beta)
            
            URM_train_ICM_all_SLIM_BPR = sps.vstack([URM_train*(1-0.6516874746588176), ICM_all.T*0.6516874746588176])
            recommender_SLIM_BPR = SLIM_BPR_Cython(URM_train_ICM_all_SLIM_BPR, recompile_cython=False, verbose=False)            
            filename_SLIM_BPR = 'SLIM_BPR_fold_number' + str(j)
            recommender_SLIM_BPR.load_model('/content/drive/MyDrive/Rec_sys_models/SLIM_BPR_saved/', file_name=filename_SLIM_BPR)


            URM_train_ICM_all = sps.vstack([URM_train, ICM_all.T])
            hybrid = ItemKNNScoresHybridNRecommender(URM_train_ICM_all, [recommender_IALS,recommender_RP3Beta, recommender_SLIM, recommender_SLIM_BPR], 4)
            hybrid.fit([alpha,beta,gamma,phi])


            
            result_dict, _ = evaluator_validation.evaluateRecommender(hybrid)
            self.MAP_vector[i]=result_dict[10]["MAP"]
            i = i+1
            j=j+1
        
        print('printing self map vector: ', self.MAP_vector)
        MAP = np.sum(self.MAP_vector)/5
        print('printing MAP: ', MAP)



        
        return MAP
    
# Execute an optimization by using an `Objective` instance.
study = optuna.create_study(direction='maximize')
study.optimize(Objective(URM_train_vector, ICM_all, Evaluator_vector), n_trials=100)

print(study.best_params)

[I 2021-01-06 11:22:23,660] A new study created in memory with name: no-name-19209ec2-00b5-4f33-9329-643f366e6d51


Recommender_Base_Class: URM Detected 73 (0.26 %) cold users.
Recommender_Base_Class: URM Detected 1 (0.00 %) cold items.
Recommender_Base_Class: Loading model from file '/content/drive/MyDrive/Rec_sys_models/MF_saved_2/MF_IALS_fold_number1'
DataIO: Folder /content/drive/MyDrive/Rec_sys_models/MF_saved_2/.temp_DataIO__MF_IALS_fold_number1/ already exists, could be the result of a previous failed save attempt or multiple saver are active in parallel. Folder will be removed.
Recommender_Base_Class: Loading complete
SLIMElasticNetRecommender: URM Detected 73 (0.26 %) cold users.
SLIMElasticNetRecommender: URM Detected 1 (0.00 %) cold items.
SLIMElasticNetRecommender: Loading model from file '/content/drive/MyDrive/Rec_sys_models/SLIM_saved/SLIM_EN_fold_number1'
SLIMElasticNetRecommender: Loading complete
RP3betaRecommender: URM Detected 73 (0.26 %) cold users.
RP3betaRecommender: URM Detected 1 (0.00 %) cold items.
RP3betaRecommender: Loading model from file '/content/drive/MyDrive/Rec_sys

[I 2021-01-06 11:24:53,466] Trial 0 finished with value: 0.07721301021838875 and parameters: {'alpha': 0.3910002862779465, 'beta': 0.4146643472559519, 'gamma': 0.3088663253957581, 'phi': 0.4540401151015483}. Best is trial 0 with value: 0.07721301021838875.


printing self map vector:  [0.08301474 0.07599715 0.07749595 0.07555302 0.07400419]
printing MAP:  0.07721301021838875
Recommender_Base_Class: URM Detected 73 (0.26 %) cold users.
Recommender_Base_Class: URM Detected 1 (0.00 %) cold items.
Recommender_Base_Class: Loading model from file '/content/drive/MyDrive/Rec_sys_models/MF_saved_2/MF_IALS_fold_number1'
Recommender_Base_Class: Loading complete
SLIMElasticNetRecommender: URM Detected 73 (0.26 %) cold users.
SLIMElasticNetRecommender: URM Detected 1 (0.00 %) cold items.
SLIMElasticNetRecommender: Loading model from file '/content/drive/MyDrive/Rec_sys_models/SLIM_saved/SLIM_EN_fold_number1'
SLIMElasticNetRecommender: Loading complete
RP3betaRecommender: URM Detected 73 (0.26 %) cold users.
RP3betaRecommender: URM Detected 1 (0.00 %) cold items.
RP3betaRecommender: Loading model from file '/content/drive/MyDrive/Rec_sys_models/RP3_saved_2/RP3_fold_number1'
RP3betaRecommender: Loading complete
ItemKNNScoresHybridNRecommender: URM Detec

[I 2021-01-06 11:26:35,596] Trial 1 finished with value: 0.07675693673635824 and parameters: {'alpha': 0.7779445549138364, 'beta': 0.4403842919908809, 'gamma': 0.18067267524249309, 'phi': 0.8742163121783099}. Best is trial 0 with value: 0.07721301021838875.


...

printing self map vector:  [0.08438629 0.07743827 0.07843042 0.07671644 0.07442955]
printing MAP:  0.0782801952409116
Recommender_Base_Class: URM Detected 73 (0.26 %) cold users.
Recommender_Base_Class: URM Detected 1 (0.00 %) cold items.
Recommender_Base_Class: Loading model from file '/content/drive/MyDrive/Rec_sys_models/MF_saved_2/MF_IALS_fold_number1'
Recommender_Base_Class: Loading complete
SLIMElasticNetRecommender: URM Detected 73 (0.26 %) cold users.
SLIMElasticNetRecommender: URM Detected 1 (0.00 %) cold items.
SLIMElasticNetRecommender: Loading model from file '/content/drive/MyDrive/Rec_sys_models/SLIM_saved/SLIM_EN_fold_number1'
SLIMElasticNetRecommender: Loading complete
RP3betaRecommender: URM Detected 73 (0.26 %) cold users.
RP3betaRecommender: URM Detected 1 (0.00 %) cold items.
RP3betaRecommender: Loading model from file '/content/drive/MyDrive/Rec_sys_models/RP3_saved_2/RP3_fold_number1'
RP3betaRecommender: Loading complete
ItemKNNScoresHybridNRecommender: URM Detect

[I 2021-01-06 14:13:59,878] Trial 98 finished with value: 0.07854667859359135 and parameters: {'alpha': 0.14437291906752261, 'beta': 0.8280826279809712, 'gamma': 0.10013493204432827, 'phi': 0.7790010729326791}. Best is trial 98 with value: 0.07854667859359135.


printing self map vector:  [0.08469594 0.0776524  0.07886075 0.07709454 0.07442977]
printing MAP:  0.07854667859359135
Recommender_Base_Class: URM Detected 73 (0.26 %) cold users.
Recommender_Base_Class: URM Detected 1 (0.00 %) cold items.
Recommender_Base_Class: Loading model from file '/content/drive/MyDrive/Rec_sys_models/MF_saved_2/MF_IALS_fold_number1'
Recommender_Base_Class: Loading complete
SLIMElasticNetRecommender: URM Detected 73 (0.26 %) cold users.
SLIMElasticNetRecommender: URM Detected 1 (0.00 %) cold items.
SLIMElasticNetRecommender: Loading model from file '/content/drive/MyDrive/Rec_sys_models/SLIM_saved/SLIM_EN_fold_number1'
SLIMElasticNetRecommender: Loading complete
RP3betaRecommender: URM Detected 73 (0.26 %) cold users.
RP3betaRecommender: URM Detected 1 (0.00 %) cold items.
RP3betaRecommender: Loading model from file '/content/drive/MyDrive/Rec_sys_models/RP3_saved_2/RP3_fold_number1'
RP3betaRecommender: Loading complete
ItemKNNScoresHybridNRecommender: URM Detec

[I 2021-01-06 14:15:43,416] Trial 99 finished with value: 0.07843798670741954 and parameters: {'alpha': 0.13718457448462285, 'beta': 0.8643677825999514, 'gamma': 0.1126773023711975, 'phi': 0.555268490351952}. Best is trial 98 with value: 0.07854667859359135.


printing self map vector:  [0.08467799 0.07754786 0.07884242 0.07671199 0.07440967]
printing MAP:  0.07843798670741954
{'alpha': 0.14437291906752261, 'beta': 0.8280826279809712, 'gamma': 0.10013493204432827, 'phi': 0.7790010729326791}
